## TROCR Test Notebook

This notebook allows to test the base model or finetuned model with a demo cropped image

It uses the cropped image/s generated by the DETR model and recognize the text in all of them

Its the second part of the main pipeline

In [ ]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import os
import torch

In [2]:
CKP_MODEL = "../finetuned/trocr/Terminus/V_2_final"
DEVICE = "cuda" # Where to load the model

In [ ]:
processor = TrOCRProcessor.from_pretrained(CKP_MODEL,device_map=DEVICE)
model = VisionEncoderDecoderModel.from_pretrained(CKP_MODEL, device_map=DEVICE)
# This configuration allows to correctly set the BOS token for inference and the Temperature and Sample strategy for the decoder
# This could be setted on the generation_config.json file, generated when saving the model, but, to avoid mismatch or errors, I overwrite this
model.generation_config.decoder_start_token_id = processor.tokenizer.bos_token_id
model.generation_config.temperature = 0.4
model.generation_config.do_sample = True

In [ ]:
# To check the generation config
model.generation_config

In [40]:
demo_image = "/home/ralvarez22/Imágenes/Capturas de pantalla/Captura desde 2025-01-19 17-54-51.png" # Remember, must be cropped

In [41]:
#Load the image with PILL
pil_image = Image.open(demo_image).convert("RGB")

In [ ]:
# Preview the image
pil_image

In [43]:
# Get the image pixels
proc_image = processor(pil_image, return_tensors="pt").pixel_values.to(DEVICE)

In [44]:
# Call the autorregresive method of the model to generate N tokens from the input image
data_generated = model.generate(proc_image)

In [ ]:
# Check the output (tokens)
data_generated

In [ ]:
# Convert from tokens to text
processor.tokenizer.decode(data_generated[0].cpu())